## 0 環境準備

In [20]:
# from google.colab import drive
# drive.mount('/content/drive')

このコードは, Google Colab上でGoogleドライブをマウントするためのものです. 

1. **`google.colab` モジュールのインポート**: `google.colab` モジュールは, Google Colab上で動作するPythonプログラムに, Googleドライブへのアクセスを提供します. 
2. **`drive` オブジェクトの作成**: `drive.mount()` 関数を呼び出すことで, `drive` オブジェクトを作成します. このオブジェクトは, Googleドライブへのアクセスを管理するために使用されます. 
3. **マウントポイントの指定**: `drive.mount()` 関数には, マウントポイントとして使用するローカルディレクトリパスを指定する引数 `mount_point` があります. このコードでは, `/content/drive` ディレクトリをマウントポイントとして指定しています. 

**2. コードの実行方法**

このコードを実行するには, 以下の手順が必要です. 

1. Google Colabを開きます. 
2. コードセルを作成し, 上記のコードを貼り付けます. 
3. コードセルを実行します. 

**3. コード実行後の状態**

コードが正常に実行されると, Googleドライブが `/content/drive` ディレクトリにマウントされます. つまり, `/content/drive` ディレクトリにアクセスすることで, Googleドライブ上のファイルやフォルダにアクセスできるようになります. 


In [8]:
!pip install transformers fugashi ipadic
!pip install demoji
!pip install neologdn

In [26]:
!wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"
!tar -zxvf ldcc-20140209.tar.gz

## 1 データの準備(jsonファイルの作成)

In [47]:
import glob
import os
import json
import re
import demoji
import neologdn
import string

このコードは, Pythonで以下のライブラリをインポートしています. 

* **`glob`**: ファイルパスのパターンマッチングを行うライブラリです. 
* **`os`**: オペレーティングシステムとのインタラクションを提供するライブラリです. 
* **`json`**: JSON形式のデータの読み書きを行うライブラリです. 
* **`re`**: 正規表現処理を行うライブラリです. 
* **`demoji`**: 絵文字の処理を行うライブラリです. 
* **`neologdn`**: 現代日本語の表記ゆらぎの処理を行うライブラリです. 
* **`string`**: 文字列処理に関わる定数や関数を提供するライブラリです. 

In [48]:
path = "./text"
text_dir = os.listdir(path)
category_list = [f for f in text_dir if os.path.isdir(os.path.join(path, f))]
print(category_list)

['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']


1. **対象ディレクトリの指定**: `/content/text` ディレクトリを `path` 変数に格納します. 
2. **ディレクトリ一覧の取得**: `os.listdir(path)` 関数を使って, `path` ディレクトリ内にあるすべてのファイルとディレクトリのリストを取得します. 
3. **サブディレクトリの抽出**: リストされた各要素に対して, `os.path.isdir()` 関数を使ってディレクトリかどうかを判定し, ディレクトリのみを `category_list` 変数に格納します. 
4. **サブディレクトリリストの出力**: `print(category_list)` 関数を使って, `category_list` 変数に格納されたサブディレクトリリストを出力します. 

**1. コードの詳細**

**1.1 `path = "/content/text"`**

この行は, 対象となるディレクトリパスを `/content/text` に設定します. 

**1.2 `text_dir = os.listdir(path)`**

この行は, `os.listdir()` 関数を使って, `path` ディレクトリ内にあるすべてのファイルとディレクトリのリストを取得し, `text_dir` 変数に格納します. 

**1.3 `category_list = [f for f in text_dir if os.path.isdir(os.path.join(path, f))]`**

この行は, 以下の処理を実行します. 

* `f for f in text_dir`: `text_dir` リスト内の各要素 `f` についてループ処理を行います. 
* `os.path.isdir(os.path.join(path, f))`: `f` を `path` と結合したパスがディレクトリかどうかを判定します. 
* `if`: 判定結果が `True` の場合のみ, `f` を `category_list` に追加します. 

この処理により, `category_list` 変数には, `path` ディレクトリ内のサブディレクトリのみが格納されます. 

**1.4 `print(category_list)`**

この行は, `print()` 関数を使って, `category_list` 変数に格納されたサブディレクトリリストを出力します. 

In [ ]:
id_category_list = []
for index, category in enumerate(category_list):
    category_dict = {"id": index, "category": category}
    id_category_list.append(category_dict)
print(f"id_category_list: {id_category_list}")


[{'id': 0, 'category': 'movie-enter'}, {'id': 1, 'category': 'it-life-hack'}, {'id': 2, 'category': 'kaden-channel'}, {'id': 3, 'category': 'topic-news'}, {'id': 4, 'category': 'livedoor-homme'}, {'id': 5, 'category': 'peachy'}, {'id': 6, 'category': 'sports-watch'}, {'id': 7, 'category': 'dokujo-tsushin'}, {'id': 8, 'category': 'smax'}]


このコードは, カテゴリとそれに対応するIDを格納した辞書のリストを作成します. 

**1. 空のリストを作成する**

最初の行 `id_category_list = []` は, `id_category_list` という名前の空のリストを作成します. このリストは, カテゴリとIDのペアを格納するために使用されます. 

**2. カテゴリをループ処理する**

2番目の行 `for index, category in enumerate(category_list)` は, `category_list` 内の各カテゴリをループ処理します. このループは, `category_list` の要素数だけ繰り返されます. 

- `index` 変数は, 現在のカテゴリの `category_list` 内のインデックスを保持します. インデックスは 0 から始まり, 各ループで 1 ずつ増えます. 
- `category` 変数は, 現在のループで処理されているカテゴリの実際の値を保持します. 

**3. カテゴリとIDを格納した辞書を作成する**

3番目の行 `category_dict = {"id": index, "category": category}` は, `category_dict` という名前の辞書を作成します. この辞書には, 2つのキーと値のペアが含まれます. 

- `"id"` キーには, 現在の `index` の値が割り当てられます. これは, このカテゴリの一意なIDを表します. 
- `"category"` キーには, `category_list` から取得した現在のカテゴリ名または値が格納されます. 

**4. 辞書をリストに追加する**

4番目の行 `id_category_list.append(category_dict)` は, 新しく作成された `category_dict` を `id_category_list` に追加します. つまり, ループの各反復で 1 つの辞書がリストに追加され, カテゴリとIDのペアのリストが効率的に構築されます. 

**5. 最終的なリストを出力する**

最後の行 `print(id_category_list)` は, 単に `id_category_list` の内容をコンソールに出力します. これにより, 作成されたカテゴリとIDの辞書のリストを確認できます. 

In [50]:
annotations_list = []
for item in id_category_list:
    file_list = glob.glob(f'text/{item["category"]}/{item["category"]}*.txt')
    for file in file_list:
        annotation_dict = {"file_name": os.path.basename(file), "label": item["id"], "category_name": item["category"]}
        annotations_list.append(annotation_dict)
print(annotations_list[:4])

[{'file_name': 'movie-enter-6361791 3.txt', 'label': 0, 'category_name': 'movie-enter'}, {'file_name': 'movie-enter-5978741.txt', 'label': 0, 'category_name': 'movie-enter'}, {'file_name': 'movie-enter-6322901.txt', 'label': 0, 'category_name': 'movie-enter'}, {'file_name': 'movie-enter-6316535 2.txt', 'label': 0, 'category_name': 'movie-enter'}]


* `annotations_list` という空のリストを作成します. 
* `id_category_list` というリストをループ処理します. 
* 各要素 `item` について, 以下の処理を実行します. 
    * `glob.glob()` 関数を使って, `text/{item["category"]}/{item["category"]}*.txt` というパターンに一致するファイルパスのリストを取得します. 
    * 取得したファイルパスリスト `file_list` をループ処理します. 
    * 各ファイルパス `file` について, 以下の情報を格納した辞書 `annotation_dict` を作成します. 
        * `file_name`: ファイル名
        * `label`: ラベル (item["id"])
        * `category_name`: カテゴリ名 (item["category"])
    * 作成した `annotation_dict` を `annotations_list` に追加します. 
* `annotations_list[:4]` を出力します. 


In [51]:
json_dict = {"category": id_category_list, "annotations": annotations_list}

**1. JSON形式の辞書の作成**

* `json_dict` という空の辞書を作成します. 
* `json_dict` に以下のキーと値を格納します. 
    * `category`: `id_category_list` の内容
    * `annotations`: `annotations_list` の内容
* 作成した `json_dict` を返します. 


In [53]:
json_save_path = "./text/dataset.json"
with open(json_save_path, mode="wt", encoding="utf-8") as f:
    json.dump(json_dict, f, indent=4)

**JSONデータの保存**

* `json_save_path` 変数に, 保存先のファイルパス `/content/text/dataset.json` を設定します. 
* `with open()` ステートメントを使って, `json_save_path` を `wt` モード (書き込みモード) と `utf-8` エンコーディングで開きます. 
* `json.dump()` 関数を使って, `json_dict` 変数に格納されたJSONデータをファイルに書き込みます. 
* 書き込み後, ファイルは自動的に閉じられます. 

## 2 Dataloader作成

In [54]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertJapaneseTokenizer
torch.manual_seed(0)

このコードは, PyTorchとBERT Japanese Tokenizerを使って機械学習のタスクを実行するための準備を行うものです. 

**1. ライブラリのインポート**

* `torch`: PyTorch本体のライブラリ
* `torch.utils.data`: PyTorch DataLoaderなどのデータ処理モジュール
* `Dataset`: PyTorch DataLoader用のデータセットクラス
* `DataLoader`: PyTorch DataLoaderクラス
* `BertJapaneseTokenizer`: BERT Japanese Tokenizer

**1.1 `import torch`**

この行は, PyTorch本体のライブラリをインポートします. 機械学習モデルの構築やデータ処理など, PyTorchの様々な機能を利用するために必要です. 

**1.2 `from torch.utils.data import Dataset, DataLoader`**

この行は, PyTorch DataLoader用のデータセットクラス `Dataset` と DataLoaderクラス `DataLoader` をインポートします. これらのクラスは, 機械学習モデルの訓練や評価に使用するデータを効率的に処理するために使用されます. 

**1.3 `from transformers import BertJapaneseTokenizer`**

この行は, BERT Japanese Tokenizerをインポートします. これは, 日本語テキストをBERTモデルで処理するために必要なトークン化を行うライブラリです. 

**1.4 `torch.manual_seed(0)`**

この行は, PyTorchの乱数シードを0に設定します. これは, 機械学習モデルの訓練結果を再現できるようにするために重要です. 乱数シードを設定することで, 異なる環境でも同じ結果を得られるようにします. 


In [55]:
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [57]:
class LivedoorDataset(Dataset):
    def __init__(self, tokenizer, text_dir=None):
        self.text_dir = text_dir
        self._load_json()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.annotations_list)

    def __getitem__(self, idx):
        text = self._get_text(self.annotations_list[idx]["category_name"], 
                              self.annotations_list[idx]["file_name"])
        encoding = self.tokenizer(text, return_tensors="pt", max_length=512, padding="max_length", truncation=True)
        encoding = {key: torch.squeeze(value) for key, value in encoding.items()}
        encoding["labels"] = self.annotations_list[idx]["label"]

        return encoding
    
    def _load_json(self):
        with open(os.path.join(self.text_dir, 'dataset.json')) as f:
            self.text_json = json.load(f)
        self.annotations_list = self.text_json["annotations"]
        
    def _get_text(self, category_name, file_name):
        file_path = os.path.join(self.text_dir, category_name, file_name)
        lines = open(file_path).read().splitlines()
        text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す. 
        text_preprocessed = self._text_preprocess(text)
        return text_preprocessed
        
    def _text_preprocess(self, text):
        # タブの消去
        text = text.translate(str.maketrans({'\n': '', '\t': '', '\r': '', '\u3000': ''}))

        # URLの消去
        text = re.sub(r'http?://[\w/:%#$&\?~\.=\+\-]+', '', text)
        text = re.sub(r'https?://[\w/:%#$&\?~\.=\+\-]+', '', text)

        # 絵文字の消去
        text = demoji.replace(string=text, repl='')

        # 文字の正規化
        text = neologdn.normalize(text)

        # 数字をすべて0に
        text = re.sub(r'\d+', '0', text)

        # 大文字を小文字に
        text = text.lower()

        # 【関連記事, 関連サイト, 関連リンク】以降の消去
        target_list = ['関連記事', '関連サイト', '関連リンク']
        for target in target_list:
          idx = text.find(target)
          text = text[:(idx-1)]

        return text

このコードは, PyTorch DataLoaders用の `LivedoorDataset` クラスを定義しています. このクラスは, Livedoorコーパスのテキストデータを読み込み, BERT Japanese Tokenizerを使ってトークン化し, PyTorchモデルで扱える形式に変換する役割を担います. 

**1. クラスの概要**

`LivedoorDataset` クラスは, 以下の情報を持ちます. 

* `text_dir`: Livedoorコーパスデータのディレクトリパス
* `tokenizer`: BERT Japanese Tokenizer
* `annotations_list`: JSONファイルから読み込んだアノテーションデータ

**2. メソッド解説**

**2.1 `__init__()` メソッド**

* コンストラクタ: LivedoorコーパスのディレクトリパスとBERT Japanese Tokenizerを受け取り, 以下の処理を実行します. 
    * `_load_json()` メソッドを呼び出して, JSONファイルからアノテーションデータを読み込みます. 
    * `self.tokenizer` に BERT Japanese Tokenizer を設定します. 

**2.2 `__len__()` メソッド**

* データセットの長さを返します. 
    * `len(self.annotations_list)` を返します. 

**2.3 `__getitem__()` メソッド**

* 指定されたインデックスのデータを取得します. 
    * `self.annotations_list[idx]` からアノテーションデータを取得します. 
    * `_get_text()` メソッドを使って, カテゴリ名とファイル名からテキストを取得します. 
    * BERT Japanese Tokenizerを使って, テキストをトークン化し, PyTorchモデルで扱える形式に変換します. 
    * 変換されたデータを辞書形式にして返します. 

**2.4 `_load_json()` メソッド**

* JSONファイルからアノテーションデータを読み込みます. 
    * `os.path.join(self.text_dir, 'dataset.json')` のパスでJSONファイルを開きます. 
    * JSONデータを `self.text_json` に格納します. 
    * `self.annotations_list` に `self.text_json["annotations"]` を設定します. 

**2.5 `_get_text()` メソッド**

* カテゴリ名とファイル名からテキストを取得します. 
    * `os.path.join(self.text_dir, category_name, file_name)` のパスでファイルを開きます. 
    * ファイルの内容を `lines` に格納します. 
    * `lines[3:]` を使って, 4行目以降のテキストのみを取得します. 
    * `_text_preprocess()` メソッドを使って, テキストの前処理を行います. 
    * 前処理されたテキストを返します. 

**2.6 `_text_preprocess()` メソッド**

* テキストの前処理を行います. 
    * タブ, URL, 絵文字, 文字の正規化, 数字の0化, 大文字の変換, 特定の文字列の削除などの処理を行います. 
    * 前処理されたテキストを返します. 

**3. まとめ**

`LivedoorDataset` クラスは, Livedoorコーパスのテキストデータを読み込み, BERT Japanese Tokenizerを使ってトークン化し, PyTorchモデルで扱える形式に変換する機能を提供します. このクラスを利用することで, Livedoorコーパスデータを機械学習モデルで効率的に処理することができます. 


In [58]:
dataset = LivedoorDataset(tokenizer, "./text")
print(len(dataset))
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [7000, len(dataset)-7000])
train_dataloader = DataLoader(train_dataset, batch_size=8)
val_dataloader = DataLoader(val_dataset, batch_size=8)
print(len(train_dataset), len(val_dataset))

26933
7000 19933


このコードは, PyTorchを使ってLivedoorコーパスデータを読み込み, 訓練データセットと検証データセットを作成するものです. 

**1. LivedoorDatasetクラスの利用**

* `dataset = LivedoorDataset(tokenizer, "/content/text")` の行では, LivedoorDatasetクラスを使って, Livedoorコーパスのテキストデータを読み込み, BERT Japanese Tokenizerを使ってトークン化し, PyTorchモデルで扱える形式に変換したデータセットを作成しています. 
* `/content/text` は, Livedoorコーパスのテキストデータが格納されているディレクトリパスです. 
* `tokenizer` は, BERT Japanese Tokenizerオブジェクトです. 

**2. データセットの長さの確認**

* `print(len(dataset))` の行では, 作成したデータセットの長さを出力しています. 
* データセットの長さは, Livedoorコーパスに含まれるテキストデータの件数です. 

**3. 訓練・検証データセットの作成**

* `train_dataset, val_dataset = torch.utils.data.random_split(dataset, [7000, len(dataset)-7000])` の行では, `random_split()` 関数を使って, 作成したデータセットをランダムに分割し, 訓練データセットと検証データセットを作成しています. 
* `[7000, len(dataset)-7000]` は, 分割する割合を指定します. この場合, データセット全体のうち7000件を訓練データセット, 残りの件数を検証データセットとしています. 

**4. DataLoaderの作成**

* `train_dataloader = DataLoader(train_dataset, batch_size=8)` の行では, 訓練データセットを使って DataLoader を作成しています. 
* `batch_size` は, バッチサイズを指定します. この場合, 1つのバッチに8件のデータが含まれます. 
* `DataLoader` は, データをミニバッチに分割し, 訓練や検証時に効率的に処理するために使用されます. 

* `val_dataloader = DataLoader(val_dataset, batch_size=8)` の行では, 検証データセットを使って DataLoader を作成しています. 
* `batch_size` は, 訓練データセットと同じ8に設定されています. 

**5. 訓練・検証データセットの件数の確認**

* `print(len(train_dataset), len(val_dataset))` の行では, 作成した訓練データセットと検証データセットの件数を出力しています. 
* 訓練データセットの件数は7000件, 検証データセットの件数は残りの件数となります. 

**6. まとめ**

このコードは, LivedoorコーパスデータをPyTorchで読み込み, 訓練データセットと検証データセットを作成する手順を示しています. このデータセットを使用して, BERTモデルなどの機械学習モデルを訓練することができます. 


## 3 モデルの学習

In [59]:
from transformers import BertForSequenceClassification

if torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=9).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

mps


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
import numpy as np
from tqdm.notebook import tqdm

EPOCH = 1

for epoch in range(EPOCH):
  model.train()
  train_loss = 0

  # Use tqdm for progress bar during training
  for i, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
    batch = {key: value.to(device) for key, value in batch.items()}
    output = model(**batch)
    loss = output.loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss += loss

  model.eval()
  labels_list, outputs_list = [], []

  # Use tqdm for progress bar during validation
  for i, batch in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
    batch = {key: value.to(device) for key, value in batch.items()}
    labels_list = np.concatenate([labels_list, batch["labels"].cpu().detach().numpy()])
    output = model(**batch)
    output = output.logits.argmax(axis=1).cpu().detach().numpy()
    outputs_list = np.concatenate([outputs_list, output])

  accuracy = sum(outputs_list == labels_list) / len(outputs_list) * 100
  print(f"epoch: {epoch + 1}, train_loss: {round(train_loss.item(), 1)}, accuracy: {round(accuracy, 1)}% {sum(outputs_list == labels_list)}/{len(outputs_list)}")


  0%|          | 0/875 [00:00<?, ?it/s]

  0%|          | 0/2492 [00:00<?, ?it/s]

KeyboardInterrupt: 

このコードは, PyTorchを使ってBERTモデルを訓練・評価するループ処理を示しています. 

**1. 訓練・評価ループ**

* `EPOCH = 10` : 訓練を繰り返す回数 (エポック数) を設定します. 
* `for epoch in range(EPOCH)` : エポック数分のループを回します. 

**2. 訓練処理**

* `model.train()` : モデルを訓練モードにします. 
* `train_loss = 0` : 訓練損失の累積変数を初期化します. 
* `for i, batch in enumerate(train_dataloader)` : 訓練データセットの DataLoader をイテレーションします. 
    * `batch` : ミニバッチのデータを辞書形式で取得します. 
    * `{key: value.to(device) for key, value in batch.items()}` : バッチ内の各データを指定したデバイス (CPU or GPU) に転送します. 
    * `output = model(**batch)` : モデルに入力データを流し, 出力を取得します. (** unpacking dictionary)
    * `loss = output.loss` : 出力から損失値を取得します. 
    * `optimizer.zero_grad()` : 勾配をゼロにします. 
    * `loss.backward()` : 誤差逆伝播 (backpropagation) を実行します. 
    * `optimizer.step()` : パラメータを更新します. 
    * `train_loss += loss` : 訓練損失を累積します. 

**3. 評価処理**

* `model.eval()` : モデルを評価モードにします. 
* `labels_list, outputs_list = [], []` : ラベルとモデルの予測結果を格納するリストを初期化します. 
* `for i, batch in enumerate(val_dataloader)` : 検証データセットの DataLoader をイテレーションします. 
    * 同様にバッチデータを取得し, デバイスに転送します. 
    * `labels_list` : 正解ラベルをリストに連結します. 
    * `output.logits.argmax(axis=1)` : モデルの出力からクラス確率 (logits) を取り出し, 最も確率の高いクラスインデックスを取得します.  (argmax)
    * `outputs_list` : モデルの予測結果をリストに連結します. 

**4. 精度計算と出力**

* `accuracy = ...` : 正解率 (accuracy) を計算します. 
* `print(f"...")` : エポック数, 訓練損失, 正解率, 分類ごとの正解数/総数を表示します. 

**5. まとめ**

このコードは, BERTモデルを訓練・評価する処理をループで実行します. 各エポックでモデルを訓練し, 検証データセットで評価を行います. 訓練と評価の結果はコンソールに出力されます. 

In [62]:
torch.save(model, "./model.pth")

## Appendix(データについて)

In [63]:
import textwrap

In [64]:
file_path = "text/movie-enter/movie-enter-6617244.txt"
txt_file = open(file_path).read()
print(txt_file)

http://news.livedoor.com/article/detail/6617244/
2012-06-04T08:00:00+0900
ノオミ・ラパスかシャーリーズ・セロンか、『プロメテウス』のワールドプレミアでドレス対決
　リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間5月31日に、ロンドンのレスター・スクウェアにて開催された。

　エリザベス女王陛下即位60周年記念に沸き、至る所に国旗が飾られているロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、会場には約1,500名のファンが集結した。

　『プロメテウス』は、人類史上最大の謎“人類の起源”を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱”が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことを——。

　出演は、『ミレニアム ドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリウッドを代表するオスカー女優シャーリーズ・セロン、『SHAME-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『X-MEN:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源”をビジュアル化した。 

　主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包まれた。ノオミ・ラパスは胸元の空いた黒いドレス、シャーリーズ・セロンは金色の髪によく似合う青のドレスで集まった人々を魅了。

In [65]:
lines = txt_file.splitlines()
print(lines)

['http://news.livedoor.com/article/detail/6617244/', '2012-06-04T08:00:00+0900', 'ノオミ・ラパスかシャーリーズ・セロンか、『プロメテウス』のワールドプレミアでドレス対決', '\u3000リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間5月31日に、ロンドンのレスター・スクウェアにて開催された。', '', '\u3000エリザベス女王陛下即位60周年記念に沸き、至る所に国旗が飾られているロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、会場には約1,500名のファンが集結した。', '', '\u3000『プロメテウス』は、人類史上最大の謎“人類の起源”を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱”が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことを——。', '', '\u3000出演は、『ミレニアム ドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリウッドを代表するオスカー女優シャーリーズ・セロン、『SHAME-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『X-MEN:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源”をビジュアル化した。 ', '', '\u3000主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包ま

In [66]:
text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す. 
print(text)

　リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間5月31日に、ロンドンのレスター・スクウェアにて開催された。

　エリザベス女王陛下即位60周年記念に沸き、至る所に国旗が飾られているロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、会場には約1,500名のファンが集結した。

　『プロメテウス』は、人類史上最大の謎“人類の起源”を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱”が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことを——。

　出演は、『ミレニアム ドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリウッドを代表するオスカー女優シャーリーズ・セロン、『SHAME-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『X-MEN:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源”をビジュアル化した。 

　主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包まれた。ノオミ・ラパスは胸元の空いた黒いドレス、シャーリーズ・セロンは金色の髪によく似合う青のドレスで集まった人々を魅了。写真撮影と数台のカメラの取材に応えた後、ファンサービスに終始していた。

　映画『プロメテウス』は、8月24日（金）全国ロードショー＜3D/2D同時上映＞

・映画『プロメテウス』オフィシャルサイト

■関連記事
・人類誕生の謎が明かさ

In [67]:
def text_preprocess(text):
    # タブの消去
    text = text.translate(str.maketrans({'\n': '', '\t': '', '\r': '', '\u3000': ''}))

    # URLの消去
    text = re.sub(r'http?://[\w/:%#$&\?~\.=\+\-]+', '', text)
    text = re.sub(r'https?://[\w/:%#$&\?~\.=\+\-]+', '', text)

    # 絵文字の消去
    text = demoji.replace(string=text, repl='')

    # 文字の正規化
    text = neologdn.normalize(text)

    # 数字をすべて0に
    text = re.sub(r'\d+', '0', text)

    # 大文字を小文字に
    text = text.lower()

    # 【関連記事, 関連サイト, 関連リンク】以降の消去
    target_list = ['関連記事', '関連サイト', '関連リンク']
    for target in target_list:
      idx = text.find(target)
      text = text[:(idx-1)]

    return text

In [68]:
preprocessed_text = text_preprocess(text)
print(preprocessed_text)

リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間0月0日に、ロンドンのレスター・スクウェアにて開催された。エリザベス女王陛下即位0周年記念に沸き、至る所に国旗が飾られているロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、会場には約0,0名のファンが集結した。『プロメテウス』は、人類史上最大の謎“人類の起源"を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱"が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことをー。出演は、『ミレニアムドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリウッドを代表するオスカー女優シャーリーズ・セロン、『shame-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『x-men:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、いま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源"をビジュアル化した。主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そして巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包まれた。ノオミ・ラパスは胸元の空いた黒いドレス、シャーリーズ・セロンは金色の髪によく似合う青のドレスで集まった人々を魅了。写真撮影と数台のカメラの取材に応えた後、ファンサービスに終始していた。映画『プロメテウス』は、0月0日(金)全国ロードショー<0d/0d同時上映>・映画『プロメテウス』オフィシャ


In [70]:
s_wrap_list = textwrap.wrap(''.join(preprocessed_text), 100)
print('\n'.join(s_wrap_list), "\n")

リドリー・スコット監督最新作『プロメテウス』のワールド・プレミアが、現地時間0月0日に、ロンドンのレスター・スクウェアにて開催された。エリザベス女王陛下即位0周年記念に沸き、至る所に国旗が飾られている
ロンドン。中心街レスター・スクウェアに、鮮やかなブルーカーペットが敷き詰められて、ワールド・プレミアは行われた。今まで一切ストーリーが明かされず、その全貌が謎に包まれたままだった本作の初披露とあって、
会場には約0,0名のファンが集結した。『プロメテウス』は、人類史上最大の謎“人類の起源"を解き明かす重大なヒントを地球上の古代遺跡で発見し、宇宙船プロメテウス号に乗って未知の惑星を訪れた科学者チームの
、想像を絶する運命を映し出す。めくるめく神秘と衝撃に彩られた探査航海の果てに、決して触れてはならない“パンドラの箱"が開いたとき、まだ何も知らない人類はすべてを目撃する。前人未踏の宇宙の彼方に、地球上
のあらゆる歴史や文明の概念さえも覆す、驚愕の真実が眠っていたことをー。出演は、『ミレニアムドラゴン・タトゥーの女』で強烈な演技を見せたノオミ・ラパス、日本でも多くの女性たちから圧倒的な支持を受けるハリ
ウッドを代表するオスカー女優シャーリーズ・セロン、『shame-シェイム-』でヴェネチア国際映画祭主演男優賞を獲得し、マグニート役で『x-
men:ファースト・ジェネレーション』を崇高なドラマにまで昇華させた俳優マイケル・ファスベンダーらが集う。圧倒的なオリジナリティに満ちた先見性と、アーティスティックな映像感覚で名高いスコット監督は、い
ま最も多くの注目を集めている彼ら豪華布陣を配し、観る者の好奇心や空想をはるかに超越した“人類の起源"をビジュアル化した。主要キャストのノオミ・ラパス、シャーリーズ・セロン、マイケル・ファスベンダー、そ
して巨匠リドリー・スコット監督が会場に登場するとファンの熱気に包まれた。ノオミ・ラパスは胸元の空いた黒いドレス、シャーリーズ・セロンは金色の髪によく似合う青のドレスで集まった人々を魅了。写真撮影と数台
のカメラの取材に応えた後、ファンサービスに終始していた。映画『プロメテウス』は、0月0日(金)全国ロードショー<0d/0d同時上映>・映画『プロメテウス』オフィシャ 

